In [1]:
import config
from pathlib import Path
OUTPUT_DIR = Path(config.OUTPUT_DIR)
DATA_DIR = Path(config.DATA_DIR)
WRDS_USERNAME = config.WRDS_USERNAME

import wrds

In [2]:
db = wrds.Connection(wrds_username=WRDS_USERNAME)

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [7]:
df = db.get_table(library='markit', table='CDS2020', columns=['date', 'ticker', 'country', 'currency', 'parspread', 'tenor'],obs = 10)
df

,date,ticker,country,currency,parspread,tenor
0,2020-01-01,A,United States,USD,0.013322,10Y
1,2020-01-01,A,United States,USD,0.013322,10Y
2,2020-01-01,A,United States,USD,0.012868,10Y
3,2020-01-01,A,United States,USD,0.012868,10Y
4,2020-01-01,A,United States,USD,0.013912,15Y
5,2020-01-01,A,United States,USD,0.013912,15Y
6,2020-01-01,A,United States,USD,0.013436,15Y
7,2020-01-01,A,United States,USD,0.013436,15Y
8,2020-01-01,A,United States,USD,0.001901,1Y
9,2020-01-01,A,United States,USD,0.001901,1Y


In [10]:
cds2001 = db.raw_sql(
    """
    SELECT
        date, ticker, parspread
    FROM 
        markit.CDS2001 a 
    WHERE
        a.tenor = '5Y' AND
        a.country = 'United States' 
    """,
    date_cols=['date']
    )
cds2001

,date,ticker,parspread
0,2001-01-09,AA,0.004483
1,2001-01-09,AA,0.004192
2,2001-01-10,AA,0.004150
3,2001-01-10,AA,0.003880
4,2001-01-11,AA,0.004150
...,...,...,...
197194,2001-06-27,VIA,0.004311
197195,2001-06-28,VIA,0.005833
197196,2001-06-28,VIA,0.004283
197197,2001-06-28,VIA,0.004283


In [16]:
cds_data = {} 
for year in range(2001, 2024):  # Loop from 2001 to 2023
    table_name = f"markit.CDS{year}"  # Generate table name dynamically
    query = f"""
    SELECT
        date, ticker, parspread
    FROM 
        {table_name} a 
    WHERE
        a.tenor = '5Y' AND
        a.country = 'United States'
    """
    cds_data[year] = db.raw_sql(query, date_cols=['date'])